In [44]:
# Copyright (c) 2024 Byeonghyeon Kim 
# github site: https://github.com/bhkim003/ByeonghyeonKim
# email: bhkim003@snu.ac.kr
 
# Permission is hereby granted, free of charge, to any person obtaining a copy of
# this software and associated documentation files (the "Software"), to deal in
# the Software without restriction, including without limitation the rights to
# use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
# the Software, and to permit persons to whom the Software is furnished to do so,
# subject to the following conditions:
 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
# FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
# COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
# IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


In [41]:
# 메인 셀

import sys
import torchvision
import os
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.datasets
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

# GPU   selection
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1,2" 


class SYNAPSE_FC_METHOD(torch.autograd.Function):
    @staticmethod
    def forward(ctx, spike_one_time, spike_now, weight, bias):
        ctx.save_for_backward(spike_one_time, spike_now, weight, bias)
        return F.linear(spike_one_time, weight, bias=bias)

    @staticmethod
    def backward(ctx, grad_output_current):
        #############밑에부터 수정해라#######
        spike_one_time, spike_now, weight, bias = ctx.saved_tensors
        
        ## 이거 클론해야되는지 모르겠음!!!!
        grad_output_current_clone = grad_output_current.clone()

        grad_input_spike = grad_weight = grad_bias = None


        if ctx.needs_input_grad[0]:
            grad_input_spike = grad_output_current_clone @ weight
        if ctx.needs_input_grad[2]:
            grad_weight = grad_output_current_clone.t() @ spike_now
        if bias is not None and ctx.needs_input_grad[3]:
            grad_bias = grad_output_current_clone.sum(0)

        return grad_input_spike, None, grad_weight, grad_bias

     
class SYNAPSE_FC(nn.Module):
    def __init__(self, in_features, out_features, trace_const1=1, trace_const2=0.7):
        super(SYNAPSE_FC, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.trace_const1 = trace_const1
        self.trace_const2 = trace_const2

        # self.weight = torch.randn(self.out_features, self.in_features, requires_grad=True)
        # self.bias = torch.randn(self.out_features, requires_grad=True)
        self.weight = nn.Parameter(torch.randn(self.out_features, self.in_features))
        self.bias = nn.Parameter(torch.randn(self.out_features))
        
    def forward(self, spike):
        # spike: [Time, Batch, Features]   
        Time = spike.shape[0]
        Batch = spike.shape[1] 
        output_current = torch.zeros(Time, Batch, self.out_features)

        # spike_detach = spike.detach().clone()
        spike_detach = spike.detach()
        spike_past = torch.zeros_like(spike_detach[0])
        spike_now = torch.zeros_like(spike_detach[0])

        for t in range(Time):
            spike_now = self.trace_const1*spike_detach[t] + self.trace_const2*spike_past
            output_current[t]= SYNAPSE_FC_METHOD.apply(spike[t], spike_now, self.weight, self.bias) 
            spike_past = spike_now

        return output_current 


class SYNAPSE_CONV_METHOD(torch.autograd.Function):
    @staticmethod
    def forward(ctx, spike_one_time, spike_now, weight, bias, stride=1, padding=1):
        ctx.save_for_backward(spike_one_time, spike_now, weight, bias, torch.tensor([stride], requires_grad=False), torch.tensor([padding], requires_grad=False))
        return F.conv2d(spike_one_time, weight, bias=bias, stride=stride, padding=padding)

    @staticmethod
    def backward(ctx, grad_output_current):
        spike_one_time, spike_now, weight, bias, stride, padding = ctx.saved_tensors
        stride=stride.item()
        padding=padding.item()
        
        ## 이거 클론해야되는지 모르겠음!!!!
        grad_output_current_clone = grad_output_current.clone()

        grad_input_spike = grad_weight = grad_bias = None


        if ctx.needs_input_grad[0]:
            grad_input_spike = F.conv_transpose2d(grad_output_current_clone, weight, stride=stride, padding=padding)
        if ctx.needs_input_grad[2]:
            grad_weight = torch.nn.grad.conv2d_weight(spike_now, weight.shape, grad_output_current_clone,
                                                      stride=stride.item(), padding=padding.item())
        if bias is not None and ctx.needs_input_grad[3]:
            grad_bias = grad_output_current_clone.sum((0, -1, -2))

        return grad_input_spike, None, grad_weight, grad_bias, None, None

     



class SYNAPSE_CONV(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, trace_const1=1, trace_const2=0.7):
        super(SYNAPSE_CONV, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.trace_const1 = trace_const1
        self.trace_const2 = trace_const2

        # self.weight = torch.randn(self.out_channels, self.in_channels, self.kernel_size, self.kernel_size, requires_grad=True)
        # self.bias = torch.randn(self.out_channels, requires_grad=True)
        self.weight = nn.Parameter(torch.randn(self.out_channels, self.in_channels, self.kernel_size, self.kernel_size))
        self.bias = nn.Parameter(torch.randn(self.out_channels))

    def forward(self, spike):
        # spike: [Time, Batch, Channel, Height, Width]   
        Time = spike.shape[0]
        Batch = spike.shape[1] 
        Channel = self.out_channels
        Height = (spike.shape[3] + self.padding*2 - self.kernel_size) // self.stride + 1
        Width = (spike.shape[4] + self.padding*2 - self.kernel_size) // self.stride + 1
        output_current = torch.zeros(Time, Batch, Channel, Height, Width)

        # spike_detach = spike.detach().clone()
        spike_detach = spike.detach()
        spike_past = torch.zeros_like(spike_detach[0])
        spike_now = torch.zeros_like(spike_detach[0])

        for t in range(Time):
            spike_now = self.trace_const1*spike_detach[t] + self.trace_const2*spike_past
            output_current[t]= SYNAPSE_CONV_METHOD.apply(spike[t], spike_now, self.weight, self.bias, self.stride, self.padding) 
            spike_past = spike_now

        return output_current 



class LIF_METHOD(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input_current_one_time, v_one_time, v_decay, v_threshold, v_reset, sg_width):
        v_one_time = v_one_time * v_decay + input_current_one_time # leak + pre-synaptic current integrate
        spike = (v_one_time >= v_threshold).float() #fire
        ctx.save_for_backward(v_one_time, torch.tensor([v_decay], requires_grad=False), 
                              torch.tensor([v_threshold], requires_grad=False), 
                              torch.tensor([v_reset], requires_grad=False), 
                              torch.tensor([sg_width], requires_grad=False)) # save before reset
        v_one_time = (v_one_time - spike * v_threshold).clamp_min(0) # reset
        return spike, v_one_time

    @staticmethod
    def backward(ctx, grad_output_spike, grad_output_v):
        v_one_time, v_decay, v_threshold, v_reset, sg_width = ctx.saved_tensors
        v_decay=v_decay.item()
        v_threshold=v_threshold.item()
        v_reset=v_reset.item()
        sg_width=sg_width.item()

        grad_input_current = grad_output_spike.clone()
        # grad_temp_v = grad_output_v.clone() # not used

        ################ select one of the following surrogate gradient functions ################
        #===========surrogate gradient function (rectangle)
        grad_input_current = grad_input_current * ((v_one_time - v_threshold).abs() < sg_width/2).float() / sg_width

        #===========surrogate gradient function (sigmoid)
        # sig = torch.sigmoid((v_one_time - v_threshold))
        # grad_input_current =  sig*(1-sig)*grad_input_current

        #===========surrogate gradient function (rough rectangle)
        # v_minus_th = (v_one_time - v_threshold)
        # grad_input_current[v_minus_th <= -.5] = 0
        # grad_input_current[v_minus_th > .5] = 0
        ###########################################################################################
        return grad_input_current, None, None, None, None, None

class LIF_layer(nn.Module):
    def __init__ (self, v_init = 0.0, v_decay = 0.8, v_threshold = 0.5, v_reset = 0.0, sg_width = 1):
        super(LIF_layer, self).__init__()
        self.v_init = v_init
        self.v_decay = v_decay
        self.v_threshold = v_threshold
        self.v_reset = v_reset
        self.sg_width = sg_width

    def forward(self, input_current):
        v = torch.full_like(input_current, fill_value = self.v_init, dtype = torch.float) # v (membrane potential) init
        post_spike = torch.full_like(input_current, fill_value = self.v_init, dtype = torch.float) # v (membrane potential) init
        # i와 v와 post_spike size는 여기서 다 같음: [Time, Batch, Channel, Height, Width] 

        Time = v.shape[0]
        for t in range(Time):
            # leaky하고 input_current 더하고 fire하고 reset까지 (backward직접처리)
            post_spike[t], v[t] = LIF_METHOD.apply(input_current[t], v[t], 
                                            self.v_decay, self.v_threshold, self.v_reset, self.sg_width) 

        return post_spike
    
    





In [44]:
torch.manual_seed(42)

# ONE CONV, ONE FC

TIME = 8
BATCH = 4
CHANNEL_PIXEL = 1
IMAGE_SIZE = 32

CONV_OUT_CHANNEL = 64
CONV_KERNEL_SIZE = 3


LIF_V_THRESHOLD = 1.2

FC_OUT_FEATURES = 10


class MY_SNN_MK1(nn.Module):
    def __init__(self):
        super(MY_SNN_MK1, self).__init__()
        self.synapse_conv = SYNAPSE_CONV(in_channels=CHANNEL_PIXEL, out_channels=CONV_OUT_CHANNEL, 
                            kernel_size=CONV_KERNEL_SIZE, stride=1, padding=1, trace_const1=1, trace_const2=0.7)
        self.lif_layer = LIF_layer(v_init=0.0, v_decay=0.8, v_threshold=LIF_V_THRESHOLD, v_reset=0.0, sg_width=1)
        self.synapse_FC = SYNAPSE_FC(in_features=CONV_OUT_CHANNEL * IMAGE_SIZE * IMAGE_SIZE, 
                        out_features=FC_OUT_FEATURES, trace_const1=1, trace_const2=0.7)

    def forward(self, spike_input):
        spike_input = self.synapse_conv(spike_input)
        spike_input = self.lif_layer(spike_input)

        spike_input = spike_input.view(TIME, BATCH, -1)
        spike_input = self.synapse_FC(spike_input)

        spike_input = spike_input.sum(axis=0)

        return spike_input




############################################################
####################### DATASET ############################
transform = transforms.Compose([transforms.Resize((32, 32)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5))])


trainset = torchvision.datasets.MNIST(root='/data2',
                                      train=True,
                                      download=True,
                                      transform=transform)

# Select a subset of the training set
subset_indices = torch.randperm(len(trainset))[:1000]
trainset = torch.utils.data.Subset(trainset, subset_indices)

testset = torchvision.datasets.MNIST(root='/data2',
                                     train=False,
                                     download=True,
                                     transform=transform)

train_loader = DataLoader(trainset,
                          batch_size =128,
                          shuffle = True,
                          num_workers =2)
test_loader = DataLoader(testset,
                          batch_size =128,
                          shuffle = False,
                          num_workers =2)
####################### DATASET END ############################
################################################################



net = MY_SNN_MK1()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr =0.001, momentum = 0.9)




for epoch in range(2):
    running_loss = 0.0
    for i ,data in enumerate(train_loader,0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print("Epoch: {}, Iter: {}, Loss: {}".format(epoch+1, i+1, running_loss/100))
            running_loss = 0.0

ValueError: optimizer got an empty parameter list

In [39]:
# lif sparsity test 

for i in np.arange(0.1, 3.0, 0.1):
     torch.manual_seed(42)

     # ONE CONV, ONE FC

     TIME = 8
     BATCH = 4
     CHANNEL_PIXEL = 3
     IMAGE_SIZE = 32

     CONV_OUT_CHANNEL = 64
     CONV_KERNEL_SIZE = 3


     LIF_V_THRESHOLD = i

     FC_OUT_FEATURES = 10




     # Create an instance of SYNAPSE_CONV
     synapse_conv = SYNAPSE_CONV(in_channels=CHANNEL_PIXEL, out_channels=CONV_OUT_CHANNEL, 
                                   kernel_size=CONV_KERNEL_SIZE, stride=1, padding=1, trace_const1=1, trace_const2=0.7)

     # Create an instance of LIF_layer
     lif_layer = LIF_layer(v_init=0.0, v_decay=0.8, v_threshold=LIF_V_THRESHOLD, v_reset=0.0, sg_width=1)

     # Create an instance of synapse_FC
     synapse_FC = SYNAPSE_FC(in_features=CONV_OUT_CHANNEL * IMAGE_SIZE * IMAGE_SIZE, 
                              out_features=FC_OUT_FEATURES, trace_const1=1, trace_const2=0.7)




     # Generate random spike input
     spike_input = torch.randn(TIME, BATCH, CHANNEL_PIXEL, IMAGE_SIZE, IMAGE_SIZE)



     # CONV
     conv_output_current = synapse_conv(spike_input)
     # print(conv_output_current.shape)
     # print(conv_output_current)


     # LIF
     lif_post_spike = lif_layer(conv_output_current)
     # print(lif_post_spike.shape)
     # print(lif_post_spike)

     # Calculate the ratio of 1's in lif_post_spike
     ones_ratio = (lif_post_spike == 1).sum() / lif_post_spike.numel()
     print("Ratio of 1's in lif_post_spike:", ones_ratio)



     # FC
     lif_post_spike_reshaped = lif_post_spike.view(TIME, BATCH, -1)
     fc_output_current = synapse_FC(lif_post_spike_reshaped)
     # print(fc_output_current.shape)
     # print(fc_output_current)

Ratio of 1's in lif_post_spike: tensor(0.4894)
Ratio of 1's in lif_post_spike: tensor(0.4814)
Ratio of 1's in lif_post_spike: tensor(0.4734)
Ratio of 1's in lif_post_spike: tensor(0.4655)
Ratio of 1's in lif_post_spike: tensor(0.4576)
Ratio of 1's in lif_post_spike: tensor(0.4497)
Ratio of 1's in lif_post_spike: tensor(0.4418)
Ratio of 1's in lif_post_spike: tensor(0.4339)
Ratio of 1's in lif_post_spike: tensor(0.4260)
Ratio of 1's in lif_post_spike: tensor(0.4182)
Ratio of 1's in lif_post_spike: tensor(0.4103)
Ratio of 1's in lif_post_spike: tensor(0.4026)
Ratio of 1's in lif_post_spike: tensor(0.3949)
Ratio of 1's in lif_post_spike: tensor(0.3871)
Ratio of 1's in lif_post_spike: tensor(0.3796)
Ratio of 1's in lif_post_spike: tensor(0.3720)
Ratio of 1's in lif_post_spike: tensor(0.3645)
Ratio of 1's in lif_post_spike: tensor(0.3570)
Ratio of 1's in lif_post_spike: tensor(0.3495)
Ratio of 1's in lif_post_spike: tensor(0.3422)
Ratio of 1's in lif_post_spike: tensor(0.3348)
Ratio of 1's 